# Exercise Session 7 - Kernel SVM

## 1 Introduction

Welcome to the 7th exercise session of CS233 - Introduction to Machine Learning.  

We will continue using scikit-learn, which can be reviewed from the SVM exercise (week 5).

Let us use it to train SVMs with feature expansion and different kernel functions.

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from plots import plot, plot_expand, plot_expand_poly, plot_mykernel
from sklearn import svm
%load_ext autoreload
%autoreload 2

## 2 Kernel SVM

Beyond the linear problem we discussed before, SVM can also solve non-linear classification problems by using kernel functions. We replace $\mathbf{x}_i$ with $\phi(\mathbf{x}_i)$, and then $\mathbf{x}_i^T\mathbf{x}_j$ with $k(\mathbf{x}_i,\mathbf{x}_j)$. The **dual form** of the SVM training problem is then given by:  
\begin{align}
    \underset{\{\alpha_i\}}{\operatorname{max}} \ \ 
    & \sum_{i=1}^N \alpha_i - \frac 1 2 \sum_{i=1}^N\sum_{j=1}^N \alpha_i\alpha_jy_iy_jk(\mathbf{x}_i,\mathbf{x}_j)  \\   
    \operatorname{subject \ to} & \ \ \sum_{i=1}^N \alpha_iy_i = 0 \\
                 & \ \ 0 \leq \alpha_i \leq C, \forall i \ \ 
\end{align}
**Questions**
   * How can you write $\tilde{\mathbf{w}}$ using the $\alpha_i$s and the function $\phi$?
   * How is $y(\mathbf{x})$ represented using the $\alpha_i$s?


**Answers**
   * $\tilde{\mathbf{w}} = \sum_{i=1}^N \alpha_iy_i\phi(x_i) $
   * Replacing the definition of $\tilde{\mathbf{w}}$ in the prediction equation yields 
     \begin{align}
       \hat{y}(\mathbf{x}) &= \tilde{\mathbf{w}}^T\phi(\mathbf{x}) + w_0 \\
                           &= \sum_{i=1}^N \alpha_iy_ik(\mathbf{x}_i,\mathbf{x}) + w_0
     \end{align}
   * The sum can be computed on the support vectors (${\cal S}$) only, i.e., 
       $\begin{align}
         \hat{y}(\mathbf{x}) & = \sum_{i \in {\cal S}} \alpha_iy_ik(\mathbf{x}_i,\mathbf{x}) + w_0
     \end{align}$
   
Have a look at the SVM function [here.](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) The main parameters you should look for are:
- Kernel Function: Linear, Polynomial and RBF 
    - Linear: `linear` $\langle {\bf x}_i, {\bf x}_j \rangle $.
    - Polynomial: `poly` $( \gamma \langle {\bf x}, {\bf x}' \rangle + c)^d $. $d$ is specified by the keyword `degree`, $c$ by `coef0`.
    - RBF: `rbf` $\exp(- \gamma \|{\bf x} - {\bf x}'\|^2)$. $\gamma$ is specified by the keyword `gamma` and must be greater than 0.
    
    Above, ${\bf x}$ and ${\bf x}'$ are data samples, and $\langle \cdot, \cdot \rangle$ indicates a dot product.
- Penalty term: C 
- Gamma: for the RBF and polynomial kernel
- Degree: for the polynomial kernel


### 2.1 Linearly-separable data

Let us look back at the linearly-separable data from exercise 5. This time, let's use an SVM again but with different kernels.

In [ ]:
from helpers import get_simple_dataset

# Get the simple dataset
X_linear, Y_linear = get_simple_dataset()
plot(X_linear, Y_linear, None, dataOnly=True)

#### - Linear SVM
Firstly, we can use a linear kernel.

In [ ]:
# Use an SVM with a linear kernel and set C=0.1
# You can also try some other C values and see what happens
clf = svm.SVC(kernel='linear', C=0.1)

# Call the fit method
clf.fit(X_linear, Y_linear)

# Plot the decision boundary and the support vectors
plot(X_linear, Y_linear, clf)


The above plot shows the decision boundary and margins of the learnt model. The circled points are the support vectors.

#### - Polynomial and RBF kernels 
We can then try to use Polynomial and RBF kernels. Have a look at the shape of the resulting decision boundary.

In [ ]:
# Use an SVM with a polynomial kernel and set C=0.1, degree=2
# You can also try some other C and degree values and see what happens

### CODE HERE ###
clf = svm.SVC(kernel='poly', C=0.1, degree=2, gamma=1.0, coef0=1.0)

clf.fit(X_linear, Y_linear)
plot(X_linear, Y_linear, clf)

In [ ]:
# Use an SVM with an RBF kernel and set C=0.1, gamma=0.1 , 
# You can also try some other C and gamma values and see what happens

### CODE HERE ###
clf = svm.SVC(kernel='rbf', C=0.1, gamma=0.1)

clf.fit(X_linear, Y_linear)
plot(X_linear, Y_linear, clf)


Since the data is linearly separable, kernel methods are unnecessary in this case. How about for non-linearly separable data?

### 2.2 Non-Linearly separable data
Until now we have worked with linearly-seperable data in the input feature space. Most of the time, this won't be the case. Let us now look at non-linear data and use non-linear kernels classify it. These kernels implicitly project the data to a higher dimensional space where the classes can be linearly separated.

In [ ]:
from helpers import get_circle_dataset

X,Y = get_circle_dataset()
plot(X,Y,None,dataOnly=True)

#### - Linear SVM
Is a linear SVM able to classify the data? 

In [ ]:
# Use a linear SVM
### CODE HERE ###
clf_linear = svm.SVC(kernel='linear', C=0.1)
    
clf_linear.fit(X, Y)
plot(X, Y, clf_linear)

Linear SVM fails on this non-linear dataset.

#### - Polynomial feature expansion

Now, we would like to perform an explicit polynomial feature expension to project the data to higher dimension so that it can be classified linearly. 

Fill in the function `expand_X()`. You should add a bias term, but **omit the interaction terms**. For example:

For $D=2$, $\text{degree_of_expansion}=2$ you have:
$$
\mathbf{x_i} = \begin{bmatrix}\mathbf{x}_i^{(1)}& \mathbf{x}_i^{(2)}\end{bmatrix}
$$
After the polynomial feature expansion, you would like to have:
$$ 
\mathbf{\phi}(\mathbf{x}_i) = \begin{bmatrix}1 & \mathbf{x}_i^{(1)} & \mathbf{x}_i^{(2)} & (\mathbf{x}_i^{(1)})^2 & (\mathbf{x}_i^{(2)})^2 \end{bmatrix}
$$

In [ ]:
def expand_X(X, degree_of_expansion):
    """  Perform degree-d polynomial feature expansion of X, 
         with bias but omitting the interaction terms
    
    Args:
        X (np.array): data, shape (N, D).
        degree_of_expansion (int): The degree of the polynomial feature expansion.
    
    Returns:
        (np.array): Expanded data of shape (N, new_D), 
                    where new_D is D*degree_of_expansion+1
    
    """
    expanded_X = np.ones((X.shape[0],1))
    ### CODE HERE ###
    for idx in range(1,degree_of_expansion+1): 
        expanded_X = np.hstack((expanded_X, X**idx))  
    return expanded_X

In [ ]:
# polynomial feature expansion without the interaction terms
degree_of_expansion = 2
expanded_X = expand_X(X, degree_of_expansion)

print("The original data has {} features.".format(X.shape[1]))
print("After degree-{} polynomial feature expansion (with bias, without interaction terms) the data has {} features.".format(degree_of_expansion,expanded_X.shape[1]))

# Use a linear SVM on the expanded features with C=10.0
### CODE HERE ###
expanded_clf = svm.SVC(kernel='linear', C=10.0)
expanded_clf.fit(expanded_X, Y)

plot_expand(X, Y, expanded_clf, degree_of_expansion)


Now, the non-linear data can be classified by a linear SVM.

#### - Polynomial and RBF Kernels 

Now, we will use polynomial and RBF kernels. First, let's implement the polynomial kernel, which is defined as 'poly' above.

- poly: $( \gamma \langle X, X' \rangle + r)^d $. $d$ is specified by keyword `degree`, $r$ by `coef0`.

In [ ]:
# Compute your own polynomial kernel
def my_poly_kernel(X, Xprime, degree=3, gamma=1.0, coef0=1.0):
    """  Compute a degree-d polynomial kernel matrix between two sets of data X and Xprime 
    Args:
        X (np.array): data, shape (N1, D).
        Xprime (np.array): data, shape (N2, D).
        degree (int): The degree of the polynomial kernel method.
    Returns:
        K (np.array): the kernel matrix from data matrices; that matrix should be an array of shape (N1, N2).    
    """
    ### CODE HERE ###
    K = (gamma * np.dot(X, Xprime.T) + coef0) ** degree   
    return K

Considering the polynomial feature expansion with **the interaction terms** by using [sklearn.preprocessing.PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html):

**Question a**  What are the differences between polynomial feature expansion and a polynomial kernel? 

**Question b**  Is the SVM trained with a linear kernel on the polynomial-expanded data the same as the SVM trained with a polynomial kernel on the original data?

Now, let's check our implementation of polynomial kernel and find out the answer to Question b.

In [ ]:
# These three SVMs should be same.

degree_of_expansion = 3

# Baseline: 
# Use an SVM with a polynomial kernel on the original features with C=10.0, gamma=1.0, coef0=1.0
### CODE HERE ###
clf = svm.SVC(kernel='poly', C=10., degree=degree_of_expansion, gamma=1.0, coef0=1.0)
clf.fit(X, Y)

plot(X, Y, clf)

# Use an SVM with you own kernel function (my_poly_kernel), which should give the same results as the previous method when using same parameters
### CODE HERE ###
    # tip: kernel=my_poly_kernel
kernel_clf = svm.SVC(kernel=my_poly_kernel, C=10.)
kernel_clf.fit(X, Y)

plot_mykernel(X, Y, kernel_clf)

# Use an SVM with a linear kernel on the expanded features with C=10.0
## Use PolynomialFeatures to generate the expanded features with the interaction terms
from sklearn.preprocessing import PolynomialFeatures
### CODE HERE ###
poly = PolynomialFeatures(degree_of_expansion)
poly_expanded_X = poly.fit_transform(X)

print("The original data has {} features.".format(X.shape[1]))
print("After degree-{} polynomial feature expansion (with bias, with interaction terms) the data has {} features.".format(degree_of_expansion,poly_expanded_X.shape[1]))

## Use an SVM with a linear kernel on the expanded features with C=10.0
### CODE HERE ###
poly_expanded_clf = svm.SVC(kernel='linear', C=10.0)
poly_expanded_clf.fit(poly_expanded_X, Y)

plot_expand_poly(X, Y, poly_expanded_clf, degree_of_expansion)


**Answer a** Polynomial feature expansion is applied on each sample while the polynomial kernel function directly computes the dot products between the polynomial-expanded features of each pair of samples in the training set. 

**Answer b** Yes, the 2 SVMs are the same.

Let's play with different settings for different kernels.

In [ ]:
# Use an SVM with a polynomial kernel of different degrees
D = 2 ** np.linspace(0, 6, num=7)
for d in D:
    ## use poly kernel with C=10., gamma=1.0, coef0=1.0 and different degrees
    ### CODE HERE ###
    clf = svm.SVC(kernel='poly', C=10., degree=d, gamma=1.0, coef0=1.0)
    
    clf.fit(X, Y)
    plot(X, Y, clf)


In [ ]:
# Use an SVM with an RBF Kernel with differen gammas
G = np.logspace(-2,2,num=5)
for g in G:
    ## use rbf kernel with C=0.1 and different gammas
    ### CODE HERE ###
    clf = svm.SVC(kernel='rbf', C=0.1, gamma=g)
    
    clf.fit(X, Y)
    plot(X, Y, clf)

**!!!** It is important to choose appropriate parameters for the kernel function.

**Question** For the given dataset, which kernel would you consider to be the best one? If you choose the polynomial kernel, which degree would be the best? If you choose the RBF kernel, which gamma would be the best?

**Answer**: Let's use Grid Search and Cross Validation techniques to find out the answer. We can apply K-Fold cross validation to different sets of parameters. We then use the mean accuracy to select the best model.

Let's first look at K-Fold cross validation for the RBF kernel.

In [ ]:
# K-Fold cross validation for searching for the parameters gamma and C of an SVM based on an RBF kernel.
from helpers import do_cross_validation, fold_indices

# seach in log space
grid_search_c = np.logspace(-4, 10, num=15)
grid_search_gamma = np.logspace(-9, 5, num=15)

#save the accuracies for all combinations of hyperparameters
grid_val = np.zeros((len(grid_search_c), len(grid_search_gamma)))

# Perform 4 fold cross validation
k_fold = 4
k_fold_ind = fold_indices(X.shape[0], k_fold)

for i, c in enumerate(grid_search_c):
    for j, g in enumerate(grid_search_gamma):
        print('Evaluating for C:{} gamma:{} ...'.format(c, g))
        
        ## call SVM with c,g as params.
        ### CODE HERE ####
        clf = svm.SVC(kernel='rbf', C=c, gamma=g)
        
        acc = np.zeros(k_fold)
        ## do cross validation
        for k in range(k_fold):
            acc[k] = do_cross_validation(clf, k, k_fold_ind, X, Y)
            
        ## fill out the grid_val by computing the mean accuracy from k_fold runs.
        ### CODE HERE ####
        grid_val[i,j] = np.mean(acc)


In [ ]:
from plots import plot_cv_result_rbf
## show all results and the best one
plot_cv_result_rbf(grid_val,grid_search_c,grid_search_gamma)
print('Best acc:{}'.format(np.max(grid_val)))
## best params
cin,gin = np.unravel_index(np.argmax(grid_val),grid_val.shape)
print('Best Params- C:{}, Gamma:{}'.format(grid_search_c[cin],grid_search_gamma[gin]))

The above heatmap shows the accuracies for different gamma and C values. The best parameter values are used on the test set.   
**Question** Is there a relation between C and gamma?   
**Hint**: Look at how increasing one value changes the other in the heatmap. 

**Answer**: A high gamma will lead to overfitting, hence a smaller C would allow for more misclassification to counteract this phenomenon. A lower gamma leads to underfitting, and one can thus use a high C  to prevent misclassifications. 

Let's now perform K-Fold cross validation for the polynominal kernel.

In [ ]:
# K-Fold cross validation for searching for the degree and parameter C of an SVM based on an RBF kernel.
from helpers import do_cross_validation, fold_indices

grid_search_c = np.logspace(-5,5,num=11)
grid_search_degree = 2 ** np.linspace(0, 8, num=9)

#save the accuracies for all combinations of hyperparameters
grid_val = np.zeros((len(grid_search_c),len(grid_search_degree)))

# Perform 4 fold cross validation
k_fold = 4
k_fold_ind = fold_indices(X.shape[0],k_fold)

for i, c in enumerate(grid_search_c):
    for j, d in enumerate(grid_search_degree):
        print('Evaluating for C:{} degree:{} ...'.format(c, d))
        
        ## call SVM with c,d as params.
        ### CODE HERE ####
        clf = svm.SVC(kernel='poly', C=c, degree=d, gamma=1.0, coef0=1.0)
        
        acc = np.zeros(k_fold)
        # do cross validation
        for k in range(k_fold):
            acc[k] = do_cross_validation(clf,k,k_fold_ind,X,Y)
            
        ## fill out the grid_val by computing the mean accuracy from k_fold runs. 
        ### CODE HERE ####
        grid_val[i,j] = np.mean(acc)
        

In [ ]:
from plots import plot_cv_result_poly
## show all results and the best one
plot_cv_result_poly(grid_val, grid_search_c, grid_search_degree)
print('Best acc:{}'.format(np.max(grid_val)))
## best params
cin,gin = np.unravel_index(np.argmax(grid_val),grid_val.shape)
print('Best Params- C:{}, Degree:{}'.format(grid_search_c[cin],grid_search_degree[gin]))